In [ ]:
from langchain_community.llms import LlamaCpp
from pprint import pprint
import pandas as pd
import numpy as np
from transformers import LlamaTokenizerFast



In [ ]:
n_gpu_layers = 41  # Metal set to 1 is enough.
n_batch = 2048/2  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# n_batch = 2048 # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="models/llama-13b-hf_q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
    streaming=True
)

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

In [ ]:
req = "Your task is to process and complete recipes that have missing data. You intelligently infer and fill in missing data based on complete examples. \
    A recipe has form of three columns, where at least one column has a value and the others can be masked. Columns:\n\nname: <NAME>\ningredients: <INGREDIENTS>\nsteps: <STEPS>\n\nContained \
    information:\n\n<NAME> is name of the recipe.\n<INGREDIENTS> consists of ingredients used in processing actions described in steps, separated by commas.\n<STEPS> are short sentences. \
    They begin with big letter and end with a dot. One step is one sentence. Steps explain what actions are needed to perform recipe in a correct manner.\n\n\
    Fill in the missing information in given recipe as accurately, logically, coherently, and truthfully as possible. It is necessary that the output is structured in the same way as shown in the examples above, where all columns contain values and the same structure.\
    This is a recipe to fill \n\nname: tilapia in mustard cream sauce\ningredients: tilapia fillets, fresh thyme, fresh ground black pepper, salt, cooking spray, chicken broth, portabella mushroom, whipping cream, dijon mustard\nsteps: <fill>"

req = "Your task is to process and complete recipes that have missing data. You intelligently infer and fill in missing data based on complete examples. \
    A recipe has form of three columns, where at least one column has a value and the others can be masked. Columns: name: <NAME> ingredients: <INGREDIENTS> steps: <STEPS> Contained \
    information: <NAME> is name of the recipe. <INGREDIENTS> consists of ingredients used in processing actions described in steps, separated by commas. <STEPS> are short sentences. \
    They begin with big letter and end with a dot. One step is one sentence. Steps explain what actions are needed to perform recipe in a correct manner. \
    Fill in the missing information in given recipe as accurately, logically, coherently, and truthfully as possible. It is necessary that the output is structured in the same way as shown in the examples above, where all columns contain values and the same structure.\
    This is a recipe to fill - name: tilapia in mustard cream sauce ingredients: tilapia fillets, fresh thyme, fresh ground black pepper, salt, cooking spray, chicken broth, portabella mushroom, whipping cream, dijon mustard. Please provide the steps filling for this recipe."

req = ""

In [ ]:
df = pd.read_csv("train.csv")

def compose(gdf):
    comp = [f"name: {e[2]} ; ingredients: {e[4]} ; preparation:{e[3]}" for e in gdf.values[:-1]]
    # print(gdf.values[-1])
    comp = comp + [f"name: {e[2]} ; ingredients: {e[4]} ; preparation:" for e in [gdf.values[-1]]]
    return " ; ".join(comp)

req = compose(df.iloc[[11, 15, 112, 122, 133, 144, 155, 8]])
# req = "The second planet in the solar system is "

In [ ]:

len(tokenizer.encode(req))

In [ ]:
llm.get_num_tokens(req)

In [ ]:
len(req.split()), req #764

In [ ]:
# reply = ""
# for token in llm(prompt, stream=True, echo=False):
#      reply += token["choices"][0]["text"]
#      if "###" in reply:
#          break

In [ ]:
"asd" + "df"

In [ ]:

# prev = ""
# res = ""
# for token in llm.stream(req, echo=False):
#     print(str(prev)+str(token))
#     res += token
#     if " name:" == prev+token:
#         break
#     prev = str(token)


# res1 = llm.invoke(req)
# res2 = llm.invoke(req)
# res3 = llm.invoke(req)
# res4 = llm.invoke(req)

In [ ]:
print(res)

In [ ]:

print(res1)
print(res2)
print(res3)
print(res4)

# print(reply)


In [ ]:
"""
Preheat oven to 350 degrees f. Spray a 9x13x2 inch baking dish with cooking spray. Place a single layer of fish in the dish. Season with thyme, pepper and salt. Sprinkle with a little more thyme. Drizzle with a little bit of the broth. Place a portabella mushroom atop each fish. Spoon the remaining broth over the fish. Top with whipping cream. Drizzle with a little dijon. Bake for 20 minutes. Serve
Spray a 9 x 13 pan with cooking spray. In a medium bowl combine whipping cream, mustard, broth, thyme, salt and black pepper. Arrange fish fillets in the pan. Pour the cream mixture over the fish. Bake uncovered at 350 for 20 minutes. Remove from the oven. In a medium bowl combine the broth, mushrooms, thyme, salt and black pepper. Arrange the mushrooms around the fish. Pour the broth over the fish. Bake uncovered at
Lightly spray a 13 x 9 inch glass baking dish with cooking spray. Place tilapia fillets in dish. Sprinkle thyme over fish. Season fish with black pepper and salt. Bake in 350f degrees oven for 10 minutes. Turn fish over. Spoon 1/2 cup of the chicken broth over fish. Bake 10 more minutes. Meanwhile, saute mushroom in remaining 3 tablespoons butter over medium heat until golden brown. Combine whipping cream and dijon must
Preheat oven to 400 degrees f. Spray a 9 x 13 baking dish with cooking spray. Place fillets in dish, season with thyme, pepper and salt. Drizzle with 2 tablespoons of olive oil. Bake until fillets flake easily with a fork 15 minutes. Meanwhile, saute portabello in remaining 2 tablespoons olive oil until browned and softened 5 minutes. Add broth, simmer 5 minutes. Stir in cream, mustard and lemon ju

"""

In [ ]:
print(res1.split("name:")[0])
print(res2.split("name:")[0])
print(res3.split("name:")[0])
print(res4.split("name:")[0])

In [ ]:
# 100 - 22s'Preheat oven to 400 degrees f (200 degrees c). Spray 2 shallow 9-inch pie plates with cooking spray. Season fillets with thyme and black pepper to taste. Arrange fillets in plates. Sprinkle with salt and top each with mushroom half. Bake 10 to 12 minutes or until fillets flake easily when tested with fork. Combine broth and cream in a small saucepan; bring to a boil over medium heat. Reduce heat to low; stir in mustard. Remove from heat. Spoon 1/2 of sauce mixture over each fillet; top with remaining sauce mixture; sprinkle with parsley. Makes 2 servings.; '
# 1 - 48s 'Preheat oven to 375 degrees f (190 degrees c). Coat fish with cooking spray; sprinkle with thyme, black pepper and salt. Place in single layer in 9x13-inch baking dish. Bake until fish flakes easily with a fork, 15 to 20 minutes. Combine chicken broth, mushroom and whipping cream in saucepan and bring to a simmer. Cook until cream is reduced to 1/4 cup; stir in mustard. Pour over fish; sprinkle with paprika. Bake until warmed through; about 5 minutes. Serve hot. Makes 4 servings.; '
# 2 - 33.9s 'Preheat oven to 350 degrees f. Coat 2 shallow baking pans with cooking spray. Season fish with thyme, black pepper and salt. Place fish on pan. Bake 10 to 15 minutes or until fish flakes easily with fork. Meanwhile combine broth, mushroom and cream in a saucepan over medium heat.; boil until mixture is reduced to about 1/3 cup. Stir in mustard and simmer 1 minute longer. Pour sauce over fish and serve.; '
# 4 - 28.4s 'Preheat oven to 375 degrees f. Lightly spray 11×7-inch glass baking dish with cooking spray. Combine thyme, pepper and 1/8 teaspoon salt in small bowl; rub onto both sides of fish fillets. Place in prepared dish. Bake 15 minutes or until fish is just opaque in center.; '
# 8 - 25.8s 'Preheat oven to 400 degrees f. Spray baking sheet with cooking spray. Arrange fish on baking sheet and sprinkle with thyme and pepper. Drizzle with 1/2 tablespoon of oil. Bake for 12 to 15 minutes or until fish flakes easily with fork. Remove fish from oven and set aside. Saute mushrooms in remaining oil until tender. Stir in broth, mustard and cream until smooth and well blended. Add fish to sauce and serve.; '
# 1gpu - 36.6s 'Preheat oven to 350 degrees f. Spray large nonstick skillet with cooking spray; heat over medium-high heat until hot. Add fillets; cook 2 minutes on each side or until browned and fish flakes easily with fork. Transfer to shallow 1 1/2-quart baking dish; sprinkle with thyme and pepper. Combine broth, mushroom and mustard in small saucepan; bring to boil over high heat; stir in cream and cook until sauce begins to thicken, stirring frequently. Pour sauce over fillets; bake uncovered 8 minutes or until fish flakes easily with fork.; '
# 8gpu - 36.6s 'Preheat oven to 350 degrees f (175 degrees c). Lightly spray a 9x13-inch baking dish with cooking spray. Season fish with thyme and pepper. Place in prepared pan; spray tops of fish with cooking spray. Bake in preheated oven 15 to 20 minutes, until fish flakes easily with fork and is opaque throughout; turn once halfway through baking. Meanwhile, heat oil over medium-high heat in a skillet. Add mushrooms; saut until they begin to brown and shrink, about 3 to 5 minutes. Stir in broth and cook until liquid is reduced by half and mushrooms are tender, about 5 minutes more. Remove from heat and let cool slightly. Whisk in mustard and cream until well blended. Pour evenly over fish; sprinkle with cheese. Bake uncovered until sauce is bubbly and cheese is melted and lightly browned, about 10 to 15 minutes longer. Serve fish and sauce warm.;\n'
# 16gpu - 22.8s 'Spray an 8-inch square glass baking dish with cooking spray; sprinkle with 1 teaspoon of thyme and 1/8 teaspoon of pepper. Place fish in dish; spray fish with cooking spray and sprinkle with 1/8 teaspoon of thyme and 1/8 teaspoon of pepper. Bake at 400 degrees f. for 8 minutes or until fish flakes easily with a fork. Meanwhile, in a medium saucepan, whisk broth and remaining thyme and 1 teaspoon of pepper until smooth; bring to a boil and cook 3 minutes or until slightly thickened. Remove from heat and stir in mustard until smooth; keep warm. Heat oven to 400 degrees f.; spray 12-inch heavy-duty non-stick skillet with cooking spray and place over medium-high heat until hot. Add mushroom, thyme and 1/8 teaspoon of pepper; saute 3 minutes or until tender and browned. Remove from skillet with slotted'
# 128gpu - 19.2s 'Preheat oven to 425 degrees f. Sprinkle tilapia with thyme and black pepper. Spray a 13x9x2 inch glass dish with cooking spray and place the tilapia in it. Bake 12 to 15 minutes or until fish flakes easily with a fork. Meanwhile, in a small saucepan whisk together broth and mushroom; cook over medium heat until mushroom is tender and mixture is reduced to 1/4 cup; stir in cream and mustard and heat until heated through. Remove from heat and serve over fish.; '
# 258gpu - 17.2s 'In a small bowl combine thyme and pepper; set aside. Place tilapia in a single layer on a baking sheet and spray with cooking spray; sprinkle with thyme mixture. Bake at 400 degrees f. for 12 minutes or until done. Heat chicken broth in a small saucepan over medium heat until hot; stir in mushrooms and simmer 2 minutes or until mushrooms are tender. Combine cream and mustard in a small bowl; stir in hot broth and mushrooms. Heat through; spoon over tilapia.; '



In [ ]:



# 12 podpowiedzi

# 1028 gpu - 17 s 'Preheat oven to 375 degrees f (190 degrees c).Spray 8x8-inch square baking dish with cooking spray; set aside.Place tilapia fillets into prepared dish; sprinkle with fresh thyme and fresh ground black pepper and salt to taste.Drizzle with cooking spray; bake in preheated oven 5 minutes on each side or until cooked through.; '

# 1028 gpu - 18.3 s 'Preheat oven to 375 degrees. Lightly spray a 9x13-inch baking dish with cooking spray. Sprinkle tilapia with thyme and pepper. Place fillets in dish. Bake tilapia in oven about 8 minutes or until fish flakes easily with a fork. Remove tilapia from oven and set aside. In a skillet over medium heat, saute mushrooms with 1 tablespoon butter and 1 teaspoon thyme until tender, about 5 minutes. Stir in broth and remaining 2 tablespoons butter. Cook until broth thickens, about 1 minute. Remove from heat and add cream and mustard. Stir until cream is well blended. Spoon mushroom mixture over tilapia and serve.; '

# 1028 gpu - 15 s 'Preheat oven to 350 degrees. Lightly coat 8x8 inch square baking dish with cooking spray. Place fillets in dish and sprinkle each with 1/4 teaspoon each of thyme and pepper.; '

# 1028 gpu - 15 s 'Spray baking sheet with cooking spray. Spread tilapia with thyme and sprinkle with black pepper and salt to taste. Bake until just cooked through and flaky, about 7 to 10 minutes. Cut each fillet into 4 strips and set aside. Preheat oven to 450 degrees f (230 degrees c). Place portabella mushroom on baking sheet and place in oven while tilapia is cooking. Cook mushroom until lightly browned and tender on the inside, about 7 to 10 minutes.; '

# 1028 gpu - 15.5 s Place fish on waxed paper and spray with cooking spray on both sides. Sprinkle with thyme and pepper. Bake at 400 degrees for 5 minutes per inch of thickness. Meanwhile combine chicken broth and portabella mushroom in a saucepan and boil until reduced by half. Add whipping cream and mustard and reduce until thickened. Drizzle over fish and garnish with thyme.; 

# 1028 gpu - 15.5 s Preheat oven to 450 degrees f (230 degrees c). Season fish with thyme and pepper and lightly coat with cooking spray. Arrange fish in a single layer on a large rimmed baking sheet. Bake in preheated oven until fish flakes easily with a fork, about 10 to 15 minutes depending on thickness of fish. Meanwhile, heat oven to 350 degrees f (175 degrees c). Melt butter in skillet and cook onion and mushroom over medium heat until onion is translucent and mushroom is cooked through, about 5 minutes. Stir in broth and mustard and cook until mixture is reduced and thickened, about 10 to 15 minutes. Stir in cream and reduce heat to low. Melt butter in a medium nonstick skillet over medium heat.

# 1028 gpu - 15.5 s Preheat oven to 400 degrees F. Spray baking sheet with cooking spray. Combine thyme and pepper. Spread tilapia evenly with thyme mixture and sprinkle with salt. Arrange fillets on baking sheet and bake for 10 minutes per inch of thickness. Meanwhile, cook portabella mushrooms in chicken broth in non-aluminum skillet over medium heat until tender. Drain and reserve broth. In small saucepan, combine broth and cream; cook over low heat until hot. Stir in mustard. Spoon mushrooms over fillets and pour sauce over all. Bake until fish flakes easily with fork.; 

# 1028 gpu - 15.5 s Preheat oven to 400 degrees F. Spray a 9x13 inch baking dish with cooking spray. Sprinkle fish with thyme and black pepper to taste. Drizzle with olive oil and arrange fillets in dish. Bake in preheated oven 15-20 minutes or until fish flakes easily with a fork. Meanwhile, in a saucepan bring broth to a simmer over medium heat. Stir in mushroom and cook until tender, about 5 minutes. Add whipping cream and mustard; whisk until combined. Spoon mushroom mixture evenly over fish and sprinkle with chopped thyme. Bake 5-10 minutes longer or until cream is thickened and sauce is reduced to desired consistency.; 


In [ ]:



# 10 podpowiedzi

# 1028 gpu - 20 s Preheat oven to 400 degrees. Combine thyme and black pepper and sprinkle over fish. Spray 9x13 inch pan with cooking spray and place fish in pan. Bake about 12 minutes, or until fish flakes easily with a fork. Remove fish from oven and turn oven off. Heat chicken broth and mushrooms in saucepan over high heat until mushrooms are tender. Reduce heat to medium and whisk in whipping cream and mustard. Stir in 1 tsp parsley and pour over fish.; 

# 1028 gpu - 20 s Preheat oven to 350 degrees f (175 degrees c). Line a baking sheet with aluminum foil or spray with cooking spray. Place mushrooms on baking sheet and roast in oven until browned and tender about 15 minutes. Remove mushrooms from oven and set aside. Heat oil in a skillet over medium-high heat; cook fish for 5 minutes on each side. Sprinkle with thyme and pepper and set aside until ready to serve. Whisk cream, mustard, broth and garlic in saucepan over medium heat until heated through. Serve fish with mushrooms and sauce on top.; 

# 1028 gpu - 20 s Preheat oven to 400 degrees f. Coat a baking pan with cooking spray. Place fish in pan, cover with foil and bake about 15 minutes or until fish is cooked through. Mix mushroom and mustard in small saucepan. Add whipping cream and heat until mixture is heated through and sauce thickens. Toss fish with sauce and serve.; 

# 1028 gpu - 20 s Preheat oven to 350 degrees F. Line a baking sheet with aluminum foil. Spray fish with cooking spray and sprinkle with thyme and pepper to taste. Place fish in a single layer on baking sheet and spray tops with cooking spray. Bake for 10 minutes or until fish flakes easily with fork. In a small sauce pan, combine chicken broth and mushroom over medium heat and bring to boil. Reduce heat to medium-low and simmer until mushroom are tender and liquid is reduced by half, about 10-15 minutes. Strain mixture into medium sized bowl, reserving liquid in pan and discarding mushroom. Add cream and mustard and whisk until well combined. Place fish in individual serving plates and spoon 1/4 cup sauce over fish. Garnish with parsley and serve immediately.; 

# 1028 gpu - 19 s Coat fish fillets with cooking spray. Combine thyme, 1/4 tsp pepper and salt in small dish. Sprinkle fish with mixture and set aside. Heat large nonstick skillet over medium heat until hot. Add fish; cook 2 minutes per side or until fish flakes easily with fork. Remove fish from skillet; keep warm. Add mushroom to skillet; saute 1 minute or until lightly browned. Add broth; cover and cook 2 minutes. Stir in cream and mustard; heat through. Serve with fish.; 

# 1028 gpu - 19 s In a skillet, heat oil and spray over medium-high heat. Season fish with thyme, black pepper and salt. Saute fish in oil until brown on both sides. Add chicken broth and mushrooms and bring to boil. Reduce heat and simmer until fish is cooked through and mushrooms are tender, 5 to 8 minutes. Add cream and mustard to pan and simmer until sauce is thickened. Serve fish over rice.; 

# 1028 gpu - 19 s Preheat oven to 400 degrees. Sprinkle tilapia fillets with thyme and black pepper. Spray a shallow dish with cooking spray and arrange fish in it. Bake fish 12-14 minutes or until done to taste. Heat broth in a saucepan over medium-high heat and bring to a boil. Add mushroom and simmer until tender. Stir in cream and mustard and cook over medium heat until heated through. Spoon sauce over fish and serve.; 

# 1028 gpu - 19 s Spray 2-quart casserole dish with cooking spray. Arrange fillets in dish. Sprinkle with thyme and black pepper and lightly coat fillets with cooking spray. Bake at 375 degrees F 35 minutes or until fish flakes easily when tested with fork. Combine broth, mushrooms, and mustard in small saucepan over medium heat. Cook and stir until mixture is thick and mushrooms are tender., stirring often. Combine cream and cornstarch in small bowl until cornstarch is dissolved; stir into mushroom mixture. Cook and stir until thickened and bubbly., stirring often. Pour over fish and sprinkle with thyme.; 


In [ ]:



# 10 podpowiedzi - inne

# 1028 gpu - 11 s Preheat oven to 350 degrees f. Coat bottom of a 9 x 13-inch pan with cooking spray. Lay fish in the bottom of the pan. In a small bowl, combine the thyme, black pepper and the 1/2 teaspoon of salt. Sprinkle the fish with the seasoning mixture. In a small saucepan, combine the chicken broth, portabella mushroom and whipping cream. Bring to a boil over medium heat. Reduce the heat to low. Simmer for about 5 minutes until the mixture is reduced by half.

# 1028 gpu - 11 s Preheat oven to 400 degrees f. Spray a 13 x 9 inch baking dish with cooking spray. Season fillets with thyme, black pepper and a pinch of salt. Place in dish and set aside. In a medium bowl combine cream, broth, mustard and shallots. Bring to a boil over medium heat. Add fillets and top with mushrooms. Bake for 15 minutes or until tilapia flakes easily with a fork. Sprinkle with parsley.; 

# 1028 gpu - 11 s Preheat oven to 400 degrees f. Spray 4 nonstick muffin cups with cooking spray. Season tilapia fillets with fresh thyme, pepper and salt. Divide fillets among cups. Bake 10 minutes. Meanwhile, combine broth, mushroom and whipping cream in a small saucepan over medium heat. Bring to a boil. Whisk in dijon mustard. Boil until reduced to 2 cups. Spoon over fillets. Bake 1 minute or until heated through. Sprinkle with parsley. Garnish

# 1028 gpu - 11 s Lightly spray fish fillets with cooking spray. Sprinkle with thyme, pepper and salt. Arrange fish in a single layer in a baking dish. Cover with foil. Bake for 15 minutes. Meanwhile, in a small saucepan over medium heat, saute mushrooms in 1 tablespoon oil until golden brown, 4 to 5 minutes. Add broth. Bring to a boil. Reduce heat and simmer until broth is reduced to about 1/4 cup. Stir in remaining oil. Stir in mustard. Heat through. Pour over fish. Serve


In [ ]:



# 7 podpowiedzi - inne

# 1028 gpu - 18.5 s Preheat oven to 350 degrees f. Spray a 2-quart casserole dish with cooking spray. Season tilapia with thyme, pepper and salt to taste. Lay fillets in dish. In a medium saucepan, heat broth until simmering. Add mushroom. Cook until just tender, about 3 minutes. Whisk in cream, mustard and 1/2 tsp. pepper. Pour sauce over fish. Cover with foil. Bake until fish flakes easily with a fork, 20 minutes. Sprinkle with parsley. Makes 4 servings.; 

# 1028 gpu - 18.5 s Heat spray in a nonstick skillet over medium high heat. Add fillets and cook for 2 minutes on each side, or until fish flakes easily with a fork. Remove fish from pan. Add broth, thyme, and pepper to pan; bring to a boil. Reduce heat to medium low and add mushrooms. Simmer 3 minutes. Stir in cream; cook 5 minutes or until slightly thickened. Spoon cream sauce over fish; sprinkle with parsley. Makes 4 servings.; 

# 1028 gpu - 18.5 s Spray a 9 x 13 glass dish with cooking spray. Place tilapia fillets in dish. Drizzle with lemon juice. Season to taste with thyme, black pepper and salt. Bake in a preheated 350 degree f oven for 15 minutes. Meanwhile, heat chicken broth in a medium saucepan over medium high heat to a boil. Boil until reduced by half, 5 to 6 minutes. Add portabella mushroom; cook for 3 minutes or until soft. Remove from heat. Add cream to the broth mixture. Stir in dijon mustard. Pour the sauce over the fillets. Bake uncovered for an additional 10 minutes. Remove from oven. Top with fresh thyme. Serve immediately.; 

# 1028 gpu - 18.5 s Preheat oven to 400 degrees. Season tilapia with thyme, black pepper and salt. Spray a shallow pan with cooking spray. Place fillets in pan. Bake for 8 to 10 minutes or until fish flakes easily. Remove from oven. In a separate pan saute shallots with chicken broth and portabella mushroom. Reduce heat. Add whipping cream and dijon mustard. Stir until sauce thickens. Spoon sauce over tilapia. Garnish with parsley.; 


In [ ]:



# 7 podpowiedzi - inne - eot token

# 1028 gpu - asd s 
# 1028 gpu - asd s 
# 1028 gpu - asd s 
# 1028 gpu - asd s 

In [ ]:

response2 = "\n\nStephen Colbert and John Oliver, two of the most popular late-night comedians, are facing off in a rap battle to end all rap battles. The stakes are high, with bragging rights on the line and the fate of the world hanging in the balance.\n\nHere's how the rap battle might go down:\n\nColbert:\nYo, I'm the king of the late-night scene,\nMy jokes are so sharp, they'll leave you serene.\nI'm the master of satire, the one they all fear,\nMy wit is so sharp, it'll leave you in tears.\n\nOliver:\nHold up, Stephen, you think you're the best?\nI've got news for you, you're just a mess.\nMy jokes are like a sword, they'll pierce your soul,\nI'm the one they call when they want to take control.\n\nColbert:\nOh really, John? You think you can take me down?\nI've got more Emmy's than you've got frowns.\nMy show's the biggest,"
response = ".\n\nStephen Colbert:  Yo, I heard you tryna come for my crown, John Oliver!\nI'm the king of late night, don't you forget!\nMy jokes are fire, my wit is sharp,\nI'm the one they call when they want to laugh hard!\n\nJohn Oliver:  Oh please, Stephen, you're just a clown,\nYour humor's stale, your jokes are worn out.\nI'm the one who brings the heat, the one who brings the pain,\nMy comedy's like a sword, it's sharp and it's gained!\n\nStephen Colbert:  You may have some clever lines, John, but you can't touch my style,\nMy fans love me, they wouldn't trade me for a while!\nI've been doing this for years, I'm a pro,\nYou're just a Brit who thinks he can flow!\n\nJohn Oliver:  Oh snap, you're going down, Stephen, you're in the zone,\nMy comedy's not just clever, it's a work of art, it's shown"